In [ ]:
pip install sqlalchemy : terminal 
!pip install sqlaclemy : jupyter cell

In [1]:
import sqlalchemy 

In [2]:
from sqlalchemy import create_engine , Column, String , Integer
#from sqlalchemy import create_engine
#from sqlalchemy import Column
#from sqlalchemy import string
#from sqlalchemy import Integer

In [3]:
from sqlalchemy.orm import declarative_base # to  create a base for our databse 

In [4]:
engine = create_engine("sqlite:///students.db",echo =True)

In [5]:
Base = declarative_base() #all the tables are inherited from this base

In [6]:
class Student(Base):
    __tablename__ = "students"       # table name in database

    id = Column(Integer, primary_key=True) # school/1  # primary key column
    name = Column(String, nullable=False)    # name column (required)
    age = Column(Integer)                    # age column
    grade = Column(String, default="A")      # grade column with default value

    def __repr__(self):
        return f"<Student(id={self.id}, name={self.name}, age={self.age}, grade={self.grade})>"
# 2,rohit , 33, a

In [7]:
# Create the "students" table inside students.db
Base.metadata.create_all(engine)


2025-08-29 14:08:37,432 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-29 14:08:37,435 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2025-08-29 14:08:37,437 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-29 14:08:37,441 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2025-08-29 14:08:37,444 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-29 14:08:37,447 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	age INTEGER, 
	grade VARCHAR, 
	PRIMARY KEY (id)
)


2025-08-29 14:08:37,448 INFO sqlalchemy.engine.Engine [no key 0.00153s] ()
2025-08-29 14:08:37,461 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
from sqlalchemy.orm import sessionmaker

In [12]:
# Create a session factory and a session object
Session = sessionmaker(bind=engine)
session = Session()


In [13]:
# Create new students (Python objects)
student1 = Student(name="Alice", age=20, grade="A")
student2 = Student(name="Bob", age=22, grade="B")

# Add them to the session
session.add(student1)
session.add(student2)

# Save (commit) changes into database
session.commit()


2025-08-29 14:10:29,844 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-29 14:10:29,850 INFO sqlalchemy.engine.Engine INSERT INTO students (name, age, grade) VALUES (?, ?, ?) RETURNING id
2025-08-29 14:10:29,852 INFO sqlalchemy.engine.Engine [generated in 0.00027s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Alice', 20, 'A')
2025-08-29 14:10:30,209 INFO sqlalchemy.engine.Engine INSERT INTO students (name, age, grade) VALUES (?, ?, ?) RETURNING id
2025-08-29 14:10:30,211 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Bob', 22, 'B')
2025-08-29 14:10:30,214 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
# Get all students
all_students = session.query(Student).all()
print("All Students:", all_students)


2025-08-29 14:11:02,681 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age, students.grade AS students_grade 
FROM students
2025-08-29 14:11:02,684 INFO sqlalchemy.engine.Engine [cached since 10.11s ago] ()
All Students: [<Student(id=1, name=Alice, age=20, grade=A)>, <Student(id=2, name=Bob, age=22, grade=B)>]


In [16]:
# Filter students younger than 21
young_students = session.query(Student).filter(Student.age < 21).all()
print("Students age < 21:", young_students)



2025-08-29 14:11:13,216 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age, students.grade AS students_grade 
FROM students 
WHERE students.age < ?
2025-08-29 14:11:13,219 INFO sqlalchemy.engine.Engine [cached since 20.64s ago] (21,)
Students age < 21: [<Student(id=1, name=Alice, age=20, grade=A)>]


In [17]:
# Get first student
first_student = session.query(Student).first()
print("First student:", first_student)

2025-08-29 14:11:20,980 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age, students.grade AS students_grade 
FROM students
 LIMIT ? OFFSET ?
2025-08-29 14:11:20,982 INFO sqlalchemy.engine.Engine [cached since 28.39s ago] (1, 0)
First student: <Student(id=1, name=Alice, age=20, grade=A)>


In [18]:
# Find Alice
alice = session.query(Student).filter_by(name="Alice").first()

# Update Alice's age
alice.age = 21

# Commit changes
session.commit()


2025-08-29 14:11:34,955 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age, students.grade AS students_grade 
FROM students 
WHERE students.name = ?
 LIMIT ? OFFSET ?
2025-08-29 14:11:34,957 INFO sqlalchemy.engine.Engine [generated in 0.00194s] ('Alice', 1, 0)
2025-08-29 14:11:34,963 INFO sqlalchemy.engine.Engine UPDATE students SET age=? WHERE students.id = ?
2025-08-29 14:11:34,965 INFO sqlalchemy.engine.Engine [generated in 0.00193s] (21, 1)
2025-08-29 14:11:34,970 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
all_students = session.query(Student).all()
print("All Students:", all_students)


2025-08-29 14:11:53,894 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-29 14:11:53,896 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age, students.grade AS students_grade 
FROM students
2025-08-29 14:11:53,898 INFO sqlalchemy.engine.Engine [cached since 61.33s ago] ()
All Students: [<Student(id=1, name=Alice, age=21, grade=A)>, <Student(id=2, name=Bob, age=22, grade=B)>]


In [20]:
# Create a student to delete
charlie = Student(name="Charlie", age=19, grade="C")
session.add(charlie)
session.commit()

# Delete Charlie
session.delete(charlie)
session.commit()


2025-08-29 14:12:03,950 INFO sqlalchemy.engine.Engine INSERT INTO students (name, age, grade) VALUES (?, ?, ?)
2025-08-29 14:12:03,953 INFO sqlalchemy.engine.Engine [generated in 0.00276s] ('Charlie', 19, 'C')
2025-08-29 14:12:03,958 INFO sqlalchemy.engine.Engine COMMIT
2025-08-29 14:12:03,969 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-29 14:12:03,975 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age, students.grade AS students_grade 
FROM students 
WHERE students.id = ?
2025-08-29 14:12:03,977 INFO sqlalchemy.engine.Engine [generated in 0.00284s] (3,)
2025-08-29 14:12:03,982 INFO sqlalchemy.engine.Engine DELETE FROM students WHERE students.id = ?
2025-08-29 14:12:03,984 INFO sqlalchemy.engine.Engine [generated in 0.00168s] (3,)
2025-08-29 14:12:03,989 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
session.close()


2025-08-29 14:16:18,937 INFO sqlalchemy.engine.Engine ROLLBACK


In [22]:
Base.metadata.create_all(engine)

2025-08-29 14:15:58,631 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-29 14:15:58,634 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2025-08-29 14:15:58,635 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-29 14:15:58,639 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
all_students = session.query(Student).all()
print("All Students:", all_students)


2025-08-29 14:16:24,840 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-29 14:16:24,842 INFO sqlalchemy.engine.Engine SELECT students.id AS students_id, students.name AS students_name, students.age AS students_age, students.grade AS students_grade 
FROM students
2025-08-29 14:16:24,843 INFO sqlalchemy.engine.Engine [cached since 332.3s ago] ()
All Students: [<Student(id=1, name=Alice, age=21, grade=A)>, <Student(id=2, name=Bob, age=22, grade=B)>]
